### In this section, I will involve a medical-domain LLM(Bio-Medical-Llama-3-8B) as an question-generator to make hypothetical questions based on the chunked docments

### Why I choose ContactDoctor/Bio-Medical-Llama-3-8B?

#### Because:

1. It is fine-tuned based on a bounch of clinical documents. So it knows this domian better than general LLM.

2. I tested three models in my test notebook(That's a mess. I will never show it to you). Result is showing Below:

#### Task 1: Generate hypothetical questions for a document and output JSON
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is out of control, no matter how I prompt it. it run very slowly!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### Task 2: Summarize the document
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is still out of control. very slow!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### So I have no any reason not to choose ContactDoctor/Bio-Medical-Llama-3-8B

#### As the model provider's citation:

@misc{ContactDoctor_Bio-Medical-Llama-3-8B, author = ContactDoctor, title = {ContactDoctor-Bio-Medical: A High-Performance Biomedical Language Model}, year = {2024}, howpublished = {https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B}, }

Bio-Medical-Llama-3-8B model is a specialized large language model designed for biomedical applications. It is finetuned from the meta-llama/Meta-Llama-3-8B-Instruct model using a custom dataset containing over 500,000 diverse entries. These entries include a mix of synthetic and manually curated data, ensuring high quality and broad coverage of biomedical topics.

The model is trained to understand and generate text related to various biomedical fields, making it a valuable tool for researchers, clinicians, and other professionals in the biomedical domain.

In [25]:
import json
import uuid
import os
import re
from typing import List
from pathlib import Path

In [26]:
# This function will load the json file to json object
def load_json_list(path: str):    
    with open(path, mode = "r", encoding="utf-8") as f:
        return json.load(f)

In [27]:
workspace_base_path = os.getcwd()
dataset_path = os.path.join(workspace_base_path, "datasets", "chunked_medicine_data.json") 
print(dataset_path)

/home/jovyan/work/datasets/chunked_medicine_data.json


In [28]:
data = load_json_list(dataset_path)

In [29]:
data[:2]

[{'doc_id': 'b2bf17e4-42dc-4647-bde3-f6314b67c650',
  'questions': ['why is Phenylephrine prescribed?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
 {'doc_id': '1ee18da3-2fa4-4

In [30]:
from huggingface_hub import login

In [31]:
with open("keys.txt") as f:
    os.environ["HF_TOKEN"] = f.read().strip()

# login using env var
login(os.environ["HF_TOKEN"])

print(f"Login Huggingface so that we can access the model")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Login Huggingface so that we can access the model


### Now I need to use a local LLM to generate hypothetical question for the content.

In [32]:
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from transformers import BitsAndBytesConfig
import torch

#### Better quantization will speed up the inference time extrordinately.

In [34]:
def best_dtype():
    if torch.cuda.is_available():
        if torch.cuda.is_bf16_supported():
            return torch.bfloat16
        else:
            return torch.float16
        
    return torch.float32

def best_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

In [35]:
print(best_dtype())

torch.bfloat16


In [36]:
# ContactDoctor/Bio-Medical-Llama-3-8B works best at bfloat16 quantization.
# In order to load a lighter LLM and still don't lose the performance.
# I will load it in 8bit and keep the dtype as bfloat16. This is a balance choise.
def load_model(model_id: str):
    bnb_cfg = None

    if best_device() == "cuda":
        bnb_cfg = BitsAndBytesConfig(
            load_in_8bit=True, # I don't want to lost LLM's performance     
            load_in_8bit_fp32_cpu_offload=False,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config = bnb_cfg,
            dtype = best_dtype(),
            device_map = "auto",                        
        )
        print(f"The bnb configuration: {bnb_cfg}")
    else: #CPU
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )

    return model

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )
# As per my test a few times, I don't think inference this LLM model with quanlization is a good idea. As it did not that good job.
# If I load it fully, it can make amazing job. So I need to comment below line and load it in normal way.
#model = load_model(model_id)  

print("Load tokenizer and base model done!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load tokenizer and base model done!


In [14]:
print(model)                    # full architecture tree (long but useful)
print(model.config)             # core hyperparameters (dims, layers, heads…)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [15]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

Device set to use cuda


In [16]:
# Wrapper normal piple with huggingfacepipeline
hug_pipe = HuggingFacePipeline(pipeline=pipe)

In [17]:
# Transfer the pipeline to Chat mode.
# Because this is a way we can use ChatPromptTemplate to make better prompt.
question_generator = ChatHuggingFace(llm=hug_pipe)

In [19]:
# LLM model is ready. 
# Here is the definition of the structure of output schema
class QuestionList(BaseModel):
    questions: List[str] = Field(
        ...
    )

In [20]:
#structured_generator = question_generator.with_structured_output(QuestionList)

parser = PydanticOutputParser(pydantic_object=QuestionList)
format_instructions = parser.get_format_instructions() # tells the model exact JSON shape
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"questions": {"items": {"type": "string"}, "title": "Questions", "type": "array"}}, "required": ["questions"]}
```


In [21]:
# it is a dynamic prompt which instruct the LLM to response the number of questions based on the length of doc.
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a medical student. You like ask questions when you read clinical documents."),
    ("user", 
     "1.From the document below, write exactly {n} unique QUESTIONS only. "
     "2.The questions you generate should be from different medical perspectives. "
     "3.Don't generate answer. "
     "4.Return ONLY a JSON object that matches this schema:\n {{\"questions\": {{\"items\": {{\"type\": \"string\"}}, \"title\": \"Questions\", \"type\": \"array\"}}}}. \n\n"
     "5.Put the JSON between <json> and </json> tags.\n\n"),    
    ("user", "Document:\n{doc}\n\n")    
])

In [22]:
prompt

ChatPromptTemplate(input_variables=['doc', 'n'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a medical student. You like ask questions when you read clinical documents.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['n'], input_types={}, partial_variables={}, template='1.From the document below, write exactly {n} unique QUESTIONS only. 2.The questions you generate should be from different medical perspectives. 3.Don\'t generate answer. 4.Return ONLY a JSON object that matches this schema:\n {{"questions": {{"items": {{"type": "string"}}, "title": "Questions", "type": "array"}}}}. \n\n5.Put the JSON between <json> and </json> tags.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='Document:\n{doc}\n\n'), additional

### Important Note:
1. Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.(Reference: https://platform.openai.com/docs/guides/structured-outputs). For now, OpenAI and Claude's LLM support Structured outputs. Huggingface's Model usually output plain text. But we can do CoT or other prompt engineering to guide the model output what we expect.

2. So Never put the whole parser.get_format_instructions()'s structions into the chat_prompt. Just print it out and copy & pasty useful part into the prompt. In this case, I copy & pasty this {"questions": {"items": {"type": "string"}, "title": "Questions", "type": "array"}} to tell LLM this what I need.

3. Never put parser in the chain. We should always use our own regex partern to extract the string we expect, then pass to the parser to do validate, otherwise it will break the process at random point.

### If we don't follow those two rules, we will get totally mess.

In [23]:
# A simple chain which will feed a prompt to the generator to produce stucture json object with questions
#chain = prompt | question_generator | parser  # returns a QuestionList object
chain = prompt | question_generator

In [24]:
# one shot testing
doc = "that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring"
n = 3
res: QuestionList = chain.invoke({
    "n": n,
    "doc": doc,
    #"format_instructions": format_instructions
})
print(res.content) 

 <json>
{"questions": ["What is the age range for the phenylephrine products mentioned in the document?", "How should one measure the dose of phenylephrine liquid medication?", "Are there any phenylephrine products that are made for adults and can be given to children?"]}
</json>


In [151]:
# Decide how many questions it need to generate based on the length of the document.
# From my experience, 500 to 2000 : 10 questions, 100 to 500 : 5 questions, less than 100 : 1 question.
# But it can be varied.
def decide_n_questions(txt: str):
    wc = len(txt.split(" "))    
    if wc > 500:
        return 10
    elif wc > 100:
        return 5
    else:
        return 1

In [ ]:
# This is very important function using regex to extract the json string from output

pattern = re.compile(r'(?P<json>\{\s*"questions"\s*:\s*\[.*?\]\s*\})', re.DOTALL) # compile it once, so that it is reusable.

def parser_questions(doc: str):    
    m = pattern.search(doc)
    if m:
        doc = m.group('json')  # -> {"questions": ["Q1?", "Q2?"]}
        # Now parse to Pydantic    
        return parser.parse(doc.strip())
    else:
        return None

In [ ]:
# Now let's iterate all medicine's docments and invoke chain to automatically generate question. 

for content in data:
    new_questions = []
    for doc in content["docs"]:
        n = decide_n_questions(doc)   
        res = chain.invoke({"n": n, "doc": doc})            
        n_q = parser_questions(res.content)
        if n_q:
            new_questions.extend(n_q.questions)
    content['questions'].extend(new_questions)


In [164]:
data[:5]

[{'doc_id': 'b2bf17e4-42dc-4647-bde3-f6314b67c650',
  'questions': ['why is Phenylephrine prescribed?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
 {'doc_id': '1ee18da3-2fa4-4

In [ ]:
save_to = os.path.join(workspace_base_path, "datasets", "medicine_data_hypotheticalquestions.json") 

out_path = Path(save_to)
out_path.parent.mkdir(parents=True, exist_ok=True)  # make folder if needed

with out_path.open("w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)